In [1]:
import os
os.chdir('../')
%pwd

'/home/dheeraj/Projects/Explainable-AI-Non-EEG'

In [2]:
from src.dataset.create_dataset import DataframePrepAllMod  
from src.constants.constants import *
from src.model.model import ConvModel, train_cp
from src.utils.cross_val import TrainTestSplitter
from src.utils.utils import *
import tensorflow as tf


2023-11-15 14:09:50.259449: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-15 14:09:50.281300: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-15 14:09:50.281321: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-15 14:09:50.281336: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-15 14:09:50.285453: I tensorflow/core/platform/cpu_feature_g

In [3]:
import argparse
import glob
import json
import os

In [5]:
def main(args):
    device = "GPU" if tf.config.list_physical_devices('GPU') else "CPU"
    num_epochs = args.e
    batch_size = args.batch
    
    # Check if dataset file exists and create it if not
    if not os.path.isfile(os.path.join(SAVED_DATASET_DIR, args.f)): 
        print("Dataset file not found!")
        print("Generating dataset file:", args.f)
        create_data_set = DataframePrepAllMod(
            subjects=SUBJECTS, modals=MODALS, modals_1=MODALS_1, modals_2=MODALS_2,
            df_hr_spo2=HR_SPO2_FILE_PATH, df_acc_temp_eda=ACC_TEMP_EDA_FILE_PATH,
            label=LABELS, frame_rate=60, max_size=360*8
        )
        data_set = create_data_set.dataframe_prep_allmod()
        data_set.to_json(os.path.join(SAVED_DATASET_DIR, args.f))

    else:
        print("Dataset file found!")
        data_set = pd.read_json(os.path.join(SAVED_DATASET_DIR, args.f))
        if args.cross_val == 'LOSO':
            train_df, test_df = TrainTestSplitter.train_test_loso(data_set, subject_index=args.subject_index)
            train_filename = f"train_data_{args.cross_val}_{args.subject_index}.pkl"
            test_filename = f"test_data_{args.cross_val}_{args.subject_index}.pkl"

        else:
            train_df, test_df = TrainTestSplitter.train_test_losego(data_set, seg_index=args.seg_index, num_seg=args.num_seg)
            train_filename = f"train_data_{args.cross_val}_{args.seg_index}_{args.num_seg}.pkl"
            test_filename = f"test_data_{args.cross_val}_{args.seg_index}_{args.num_seg}.pkl"

        train_path = os.path.join(SAVED_DATASET_DIR, train_filename)            
        test_path = os.path.join(SAVED_DATASET_DIR, test_filename)

        save_dataframe_to_pickle(train_df, train_path) 
        save_dataframe_to_pickle(test_df, test_path)

    # Prepare data for training and validation 
    train_data, train_label = dataframe_to_array(train_df)
    train_data, train_label, val_data, val_label = preprocess_train_val(train_data, train_label)
    print("train set size:", len(train_data))
    print("val set size:", len(val_data))
    

    # Model training
    input_shape = train_data.shape[1:]

    if args.cross_val == 'LOSO':
        model_path = os.path.join(SAVED_MODEL_DIR, f"model_{args.cross_val}_{args.subject_index}.h5")
    else:
        model_path = os.path.join(SAVED_MODEL_DIR, f"model_{args.cross_val}_{args.seg_index}_{args.num_seg}.h5")

    model = ConvModel(input_shape=input_shape, nb_classes=len(LABELS))
    optimizer = tf.keras.optimizers.Adam(learning_rate=args.lr)
    train_cp(model, optimizer, train_data, train_label, val_data, val_label, epochs=args.e, batch_size=args.batch, model_path=model_path)
    print("Training Finished")
        





SyntaxError: invalid syntax (1517588486.py, line 6)

In [4]:
def train_cp(model, x_train, y_train, x_val, y_val, epochs, batch_size, path):
    """
    Trains the ConvModel model.

    Parameters:
    - model (keras.Model): ConvModel model.
    - x_train (np.ndarray): Training data.
    - y_train (np.ndarray): Training labels.
    - x_val (np.ndarray): Validation data.
    - y_val (np.ndarray): Validation labels.
    - epochs (int): Number of epochs.
    - batch_size (int): Batch size.
    - path (str): Path to save the model checkpoints.

    Returns:
    - keras.callbacks.History: History of the training process.
    """
    lr_scheduler = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model_checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath=path, 
                                                                monitor='val_accuracy', save_weights_only=True, 
                                                                mode='max', save_best_only=True)
    
    print('Training...')

    history = model.model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=epochs, batch_size=batch_size, 
                              callbacks=[lr_scheduler, early_stopping, model_checkpoint_callback])

    # Save the model checkpoints
    if not os.path.exists(path):
        os.makedirs(path)
    
    model.model.save(path)
    print('Model saved')

In [8]:
ROOT_DIR = os.path.join(os.getcwd())
SAVE_DATASET_DIR = os.path.join(ROOT_DIR, 'data', 'created_dataset', 'processed_dataframe_allmod_8Hz.pkl')
loaded_df = load_dataframe_from_pickle(SAVE_DATASET_DIR)

In [9]:
loaded_df

,Data,Subject,Label,Label_ori
0,"[[89.0, 97.0, 30.1, 0.083, 0.76, -0.87, -0.1],...",Subject_1,0,Relax_1
1,"[[71.86072423398329, 97.0, 30.8, 0.083, 0.72, ...",Subject_1,0,Relax_1
2,"[[69.0, 95.0, 31.2, 0.087, 0.14228412256267461...",Subject_1,0,Relax_1
3,"[[70.41782729805016, 95.0, 31.6, 0.087, 0.15, ...",Subject_1,0,Relax_1
4,"[[76.44289693593313, 96.0, 31.8, 0.087, 0.1554...",Subject_1,0,Relax_1
...,...,...,...,...
835,"[[77.0, 94.0, 34.5, 2.3835766016713094, 0.8621...",Subject_9,3,EmotionalStress
836,"[[74.0, 95.0, 34.3, 2.237, -0.1770194986072419...",Subject_9,3,EmotionalStress
837,"[[75.0, 96.0, 34.1, 1.77, -0.17, -0.9905292479...",Subject_9,3,EmotionalStress
838,"[[80.0, 96.40947075208913, 34.1, 1.43998050139...",Subject_9,3,EmotionalStress


In [6]:
cross_val = TrainTestSplitter(subjects=SUBJECTS, labels=LABELS)
train_df, test_df = cross_val.train_test_loso(df=loaded_df, subject_index=0)

In [7]:
nb_classes = np.unique(train_df['Label'])

In [8]:
train_data, train_label = dataframe_to_array(train_df)
train_data, train_label, val_data, val_label = preprocess_train_val(train_data, train_label)
    

In [19]:
input_shape = train_data.shape[1:]
model = ConvModel(input_shape=input_shape, nb_classes=4)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.8)
model.model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [10]:
SAVED_MODEL_DIR = os.path.join(ROOT_DIR, 'saved_models')

In [11]:
model_path = os.path.join(SAVED_MODEL_DIR, f"model_LOSO_0.h5")

In [20]:
train_cp(model, train_data, train_label, val_data, val_label, epochs=100, batch_size=64, path=model_path)

Training...
Epoch 1/100


2023-11-14 13:44:45.250766: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_2/dropout_2/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


70/70 [==============================] - 4s 39ms/step - loss: 1.0650 - accuracy: 0.5998 - val_loss: 1.2451 - val_accuracy: 0.5605 - lr: 0.0010
Epoch 2/100
70/70 [==============================] - 3s 37ms/step - loss: 0.9250 - accuracy: 0.6421 - val_loss: 1.1864 - val_accuracy: 0.5605 - lr: 0.0010
Epoch 3/100
70/70 [==============================] - 3s 37ms/step - loss: 0.8592 - accuracy: 0.6632 - val_loss: 1.2316 - val_accuracy: 0.5605 - lr: 0.0010
Epoch 4/100
70/70 [==============================] - 3s 37ms/step - loss: 0.7909 - accuracy: 0.6943 - val_loss: 1.3571 - val_accuracy: 0.5605 - lr: 0.0010
Epoch 5/100
70/70 [==============================] - 3s 37ms/step - loss: 0.7546 - accuracy: 0.7090 - val_loss: 1.3288 - val_accuracy: 0.5616 - lr: 0.0010
Epoch 6/100
70/70 [==============================] - 3s 37ms/step - loss: 0.7402 - accuracy: 0.7146 - val_loss: 0.9145 - val_accuracy: 0.6352 - lr: 0.0010
Epoch 7/100
70/70 [==============================] - 3s 37ms/step - loss: 0.7294 -

/home/dheeraj/miniconda3/envs/xaieeg/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
